In [1]:
import tensorflow as tf
# from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
# import os
import time
# import pickle

In [2]:
from names import check, raw_data, vectorize, seq_model

### Read the data

First, look in the text:

In [3]:
raw = raw_data.load_raw("../data/names.txt")
text = raw_data.join_raw(raw)
vocab = vectorize.extract_vocab(text)

print(f'Length of text: {len(text)} characters')

Length of text: 651954 characters


In [4]:
text[:300]

'Adray Lasbard                                                                                       \nAlbel Nox                                                                                           \nCliff Fittir                                                                                      '

In [5]:
# Take a look at the first 250 characters in text
print(text[:250])

Adray Lasbard                                                                                       
Albel Nox                                                                                           
Cliff Fittir                                    


In [6]:
# The unique characters in the file
vocab = vectorize.extract_vocab(text)
print(f'{len(vocab)} unique characters')

78 unique characters


## Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation. 

The `preprocessing.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [7]:
example_texts = ['abcdefg', 'xyz']

chars = vectorize.unicode_split(example_texts)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [8]:
vectorator = vectorize.Vectorator(vocab)

It converts form tokens to character IDs:

In [9]:
ids = vectorator.ids_from_chars(chars)
ids

<tf.RaggedTensor [[45, 46, 47, 48, 49, 50, 51], [68, 69, 70]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `preprocessing.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `preprocessing.StringLookup` layer so that the `[UNK]` tokens is set the same way.

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [10]:
chars = vectorator.chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [11]:
ds = vectorize.DataSet(text)

The `batch` method lets you easily convert these individual characters to sequences of the desired size.

For training you'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. 

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [12]:
vectorize.split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [13]:
# Length of the vocabulary in chars
# vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = [1024, 768]

In [14]:
model = seq_model.SeqModel(vocab,
        embedding_dim=embedding_dim,
        rnn_units=rnn_units)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](images/text_generation_training.png)

Note: For training you could use a `keras.Sequential` model here. To  generate text later you'll need to manage the RNN's internal state. It's simpler to include the state input and output options upfront, than it is to rearrange the model architecture later. For more details see the [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [15]:
for input_example_batch, target_example_batch in ds.dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 79) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [16]:
model.summary()

Model: "seq_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  20224     
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
gru_1 (GRU)                  multiple                  4133376   
_________________________________________________________________
dense (Dense)                multiple                  60751     
Total params: 8,152,655
Trainable params: 8,152,655
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [17]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [18]:
sampled_indices

array([12, 59, 19, 22, 42, 46, 32, 62, 69, 30, 23, 14, 33, 65, 66, 62,  5,
       29, 25,  5, 36, 36, 78, 59, 28, 29, 73, 30, 59, 77, 59, 46, 38, 21,
       41, 66, 34, 77, 77, 46, 57, 16, 46, 66, 11,  8, 19, 69, 74,  5, 75,
       43, 69, 73, 38, 32,  9, 35, 13, 22, 40,  5, 70, 55,  0, 39, 19, 78,
       31, 22, 18, 12, 26, 52, 51, 39, 27, 35, 39, 58, 21, 38, 52, 56, 18,
       77, 20,  4, 35, 72, 29, 63, 44, 70, 66, 70, 32, 25, 23, 77],
      dtype=int64)

Decode these to see the text predicted by this untrained model:

In [19]:
print("Input:\n", ds.text_from_ids(np.array([input_example_batch[0]])))
print()
print("Next Char Predictions:\n", ds.text_from_ids(np.array([sampled_indices])))

Input:
 ['Bwagi                                                                                               ']

Next Char Predictions:
 ["3oADXbNryLE6Ouvr,KG,RRīoJKéLoāobTCWvPāābm8bv2/Ayí,óYyéTN0Q5DV,zk[UNK]UAīMD?3HhgUIQUnCThl?āB'QáKsZzvzNGEā"]


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [20]:
example_batch_loss = model.loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 79)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.371028


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [21]:
tf.exp(mean_loss).numpy()

79.12492

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [22]:
model.compile_model()

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [23]:
chk = check.CheckPoint("jrpg_model")

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [24]:
EPOCHS = 20

In [25]:
history = model.fit_with_chk(ds, checkpoint=chk, epochs=EPOCHS)

Epoch 1/20
100/100 [==============================] - 10s 71ms/step - loss: 0.4953
Epoch 2/20
100/100 [==============================] - 8s 70ms/step - loss: 0.3468
Epoch 3/20
100/100 [==============================] - 7s 71ms/step - loss: 0.3069
Epoch 4/20
100/100 [==============================] - 8s 71ms/step - loss: 0.2850
Epoch 5/20
100/100 [==============================] - 7s 71ms/step - loss: 0.2859
Epoch 6/20
100/100 [==============================] - 8s 72ms/step - loss: 0.2760
Epoch 7/20
100/100 [==============================] - 8s 72ms/step - loss: 0.2722
Epoch 8/20
100/100 [==============================] - 8s 72ms/step - loss: 0.2726
Epoch 9/20
100/100 [==============================] - 8s 73ms/step - loss: 0.2676
Epoch 10/20
100/100 [==============================] - 8s 77ms/step - loss: 0.2643
Epoch 11/20
100/100 [==============================] - 8s 72ms/step - loss: 0.2572
Epoch 12/20
100/100 [==============================] - 8s 72ms/step - loss: 0.2487
Epoch 13/20


In [26]:
one_step_model = seq_model.OneStep(model)

In [27]:
chk.save(model)

In [28]:
def generate_names(stem="", n=1):
    names = []
    for _ in range(n):
        states = None
        next_char = [stem + ch for ch in "QWERTYUIOPASDFGHJKLZXCVBNM"]
        next_char = tf.constant(next_char)
        result = [next_char]

        for n in range(40):
          next_char, states = one_step_model.generate_one_step(next_char, states=states)
          result.append(next_char)

        result = tf.strings.join(result)
        names += [" ".join([a.strip() for a in name.decode("utf-8").split() if len(a)]) for name in result.numpy()]
    return names

In [29]:
generate_names(n=10)

['Quinnust falrryn',
 'Welch Vineyard',
 "E'Gahamemicc",
 'Rara Sith',
 'Titani',
 'Yo2Rk',
 'Ultin',
 'Irscla',
 'Owwer',
 'Pupe',
 'Ashor',
 'Spirit of the',
 'Dordea',
 'Furyyn Chack',
 'Great Fairyon',
 'Hesten',
 'Juke',
 'Kelom',
 'Lucha',
 'Zopp-Solk-',
 'Xed',
 'Cocoto',
 'Vaniel apparan',
 'Balther',
 'Nagi Naka',
 'Momino',
 'Quennart',
 'Winry Mideller',
 'Edgoror IXenorce',
 'Ruddle Crispin',
 'Tats',
 'Yukino Shiephe',
 'Ulbe',
 'Isoda Aurumbr',
 "O'ako Yama",
 'Puffy',
 'Algee Rorba',
 'Santa',
 'Dornee S.S. Tester',
 'Fenrut',
 'Gomper',
 'Hartwrah',
 'Joron',
 "K'rhid Tia",
 'Lilt Sama',
 'Zake',
 'Xook midi',
 'Cid Ornilis',
 'Vither',
 'Bugube Gente',
 'Nackyus Lemuin',
 'Medelich',
 'Quan',
 'Welwan',
 'Ellart',
 'Runa',
 'Torbenk Baye',
 'Yyus Leanced',
 'Ul Uldard',
 'Iskariaet',
 'Osman',
 'Prometa',
 'Asura',
 'Spiriton',
 'Dr. Kodek',
 'Furche',
 'Gadó',
 'Han',
 'Jack Heat',
 "Kirva's dreimer",
 'Laura',
 "Zekko's",
 'Xeus',
 'Cassai',
 'Velari',
 'Bosc Knon',


In [30]:
generate_names('Joby of ', n=10)

['Joby of Queen on Soldier',
 'Joby of Widees',
 'Joby of Esthen',
 'Joby of ResLeld',
 'Joby of Tingewss',
 'Joby of Yee',
 'Joby of Underlow Pronsol Fontes',
 'Joby of Ikt',
 'Joby of Oben',
 'Joby of Pid Flate',
 'Joby of Azbel',
 'Joby of Seller',
 'Joby of Dilns and Flo',
 'Joby of Finisblly',
 'Joby of Ginds ote',
 'Joby of Hewlin',
 'Joby of Junsiblang',
 'Joby of Kind',
 'Joby of Linchs',
 'Joby of Zeeq',
 'Joby of Xol Borls',
 'Joby of Cornelia',
 'Joby of Vestenna',
 'Joby of Bebld',
 'Joby of Neus and Foor',
 'Joby of Migus',
 'Joby of Qeeber',
 'Joby of Windsagen',
 'Joby of Eblent',
 'Joby of Robel',
 'Joby of Telld Ferole',
 'Joby of Ye Tildersan',
 'Joby of Ui',
 'Joby of Inchors',
 'Joby of Oote',
 'Joby of Pille',
 'Joby of Abbono',
 'Joby of Simigo',
 'Joby of Delle',
 'Joby of Fores',
 'Joby of Geller',
 'Joby of Hinder',
 'Joby of Jors',
 'Joby of Kisher',
 'Joby of Link',
 'Joby of Zelde',
 'Joby of XnM Mirgge',
 'Joby of Cornel',
 'Joby of Vildirgan',
 'Joby of Be

In [31]:
generate_names('Joby the ')

['Joby the Quen',
 'Joby the Wobrien',
 'Joby the Eoveim',
 'Joby the Reuviller',
 'Joby the Tynovos',
 'Joby the Yeog',
 'Joby the Urohirge',
 'Joby the Ie',
 "Joby the Ool'koin",
 'Joby the Pey',
 'Joby the Arventus',
 'Joby the Selegron',
 'Joby the Deepblax',
 'Joby the Febooth',
 'Joby the Gehste',
 'Joby the Haiden',
 'Joby the Jeussinna',
 'Joby the Koev',
 'Joby the Leusicinc',
 'Joby the Zoter',
 'Joby the X-Oovein',
 'Joby the Coutper',
 'Joby the Vonseake',
 'Joby the Bene',
 'Joby the Naki',
 'Joby the Meidius']

In [32]:
place = []
for name in generate_names():
    name = name.split()[0]
    for long_name in generate_names(name + " of "):
        print(long_name)
        place.append(long_name.split()[2])

Quina of Quen
Quina of Wald
Quina of Ezbbl
Quina of Ranson
Quina of Tirt
Quina of Yent
Quina of Untett Aptrelt
Quina of Itters
Quina of Obrir
Quina of Pint
Quina of Azelle
Quina of Sprirt
Quina of Dartess
Quina of Fanter
Quina of Garller
Quina of Hertlar
Quina of Jink
Quina of Kira
Quina of Leut
Quina of Zell
Quina of X Quen
Quina of Cardus
Quina of Van
Quina of Burmec
Quina of Nelta
Quina of Mart
Wandenigun of Qeen walds
Wandenigun of Wlane
Wandenigun of Ellha
Wandenigun of Resole
Wandenigun of Tewral
Wandenigun of Yugian
Wandenigun of Ugrello
Wandenigun of Igedus
Wandenigun of Oldirg Owner
Wandenigun of Pengagi and Four
Wandenigun of Anguri
Wandenigun of Sigglas
Wandenigun of Derno
Wandenigun of Farel
Wandenigun of Granssann
Wandenigun of Hinasa
Wandenigun of Jecron
Wandenigun of Kenny
Wandenigun of Lesurage
Wandenigun of Zurness Gorl
Wandenigun of Xeg
Wandenigun of Chotan
Wandenigun of Voglo
Wandenigun of Bage
Wandenigun of Nargess
Wandenigun of Man
Ezy of Queen
Ezy of Wumce
Ezy of 

Lohna of Quren
Lohna of Wleur gneth
Lohna of Elfhen
Lohna of Ring Flal
Lohna of Tirh
Lohna of Yuru
Lohna of Untel
Lohna of Icbrarc
Lohna of Okk Sibling
Lohna of Perma
Lohna of Ageny
Lohna of Shei
Lohna of Dla Farren
Lohna of Frat
Lohna of Girl
Lohna of Higas
Lohna of Jug Forrent
Lohna of Knrif Fleurest
Lohna of Leruk
Lohna of Zaris
Lohna of XV Leon ape
Lohna of Crararde
Lohna of Vark
Lohna of Bare
Lohna of Narg
Lohna of Mari
Zephor of Qeal
Zephor of Wlfig
Zephor of Elthef
Zephor of Rev
Zephor of Teller
Zephor of Yelen
Zephor of Underrolps
Zephor of Insect
Zephor of Oldor
Zephor of Pilla
Zephor of Allen
Zephor of Shliepte
Zephor of Delis Hilase
Zephor of Fishem
Zephor of Gilds
Zephor of Hynes
Zephor of Jean
Zephor of Kellos
Zephor of Lights
Zephor of Zelda
Zephor of XV Tenkian
Zephor of Cyvess
Zephor of Vigher
Zephor of Blindos
Zephor of Nesto
Zephor of Misaing
Xande of Queen
Xande of Wanter
Xande of Elfhe
Xande of Reles
Xande of Tian
Xande of Yabutan
Xande of Ullen Boes Alcheim
Xande o

In [33]:
place

['Quen',
 'Wald',
 'Ezbbl',
 'Ranson',
 'Tirt',
 'Yent',
 'Untett',
 'Itters',
 'Obrir',
 'Pint',
 'Azelle',
 'Sprirt',
 'Dartess',
 'Fanter',
 'Garller',
 'Hertlar',
 'Jink',
 'Kira',
 'Leut',
 'Zell',
 'X',
 'Cardus',
 'Van',
 'Burmec',
 'Nelta',
 'Mart',
 'Qeen',
 'Wlane',
 'Ellha',
 'Resole',
 'Tewral',
 'Yugian',
 'Ugrello',
 'Igedus',
 'Oldirg',
 'Pengagi',
 'Anguri',
 'Sigglas',
 'Derno',
 'Farel',
 'Granssann',
 'Hinasa',
 'Jecron',
 'Kenny',
 'Lesurage',
 'Zurness',
 'Xeg',
 'Chotan',
 'Voglo',
 'Bage',
 'Nargess',
 'Man',
 'Queen',
 'Wumce',
 'Ekker',
 'Rem',
 'Trom',
 'Yernall',
 'Uuger',
 'Ickras',
 'Oba',
 'Pheffall',
 'Allen',
 'Sage',
 'Damine',
 'Fleule',
 'Gley',
 'Hyrule',
 'Julles',
 'Kempre',
 'Lack',
 'Zres',
 'Xex',
 'Curil',
 'Villa',
 'BlfFord',
 'Nuck',
 'Maru',
 'Qeten',
 'Wlate',
 'Eulmore',
 'Rirste',
 'Tintworw',
 'Yen',
 'Upur',
 'Ilkonj',
 'Ondori',
 'Poulima',
 'Aamu',
 'Scrent',
 'Dinter',
 'Fabul',
 'Gyrus',
 'Hane',
 'Jean',
 'King',
 'Lith',
 'Zorde'

In [34]:
G = list(filter(lambda name: name[0] == "G", place))
G.sort()
G

['Gards',
 'Garller',
 'Garlss',
 'Gaster',
 'Geller',
 'Ghef',
 'Gieto',
 'Gilds',
 'Ginder',
 'Gindwents',
 'Gindwertsyn',
 'Gira',
 'Girds',
 'Girl',
 'Girns',
 'Gley',
 'Granssann',
 'Grass',
 'Gray',
 'Grimmas',
 'Gudde',
 'Gulle',
 'Gurde',
 'Gurdy',
 'Gyrla',
 'Gyrus']

In [35]:
np.random.choice(place, size=10, replace=False)

array(['Geller', 'Kinners', 'XI', 'Astral', 'Gilds', 'Aalorus', 'Zedda',
       'Leut', 'Man', 'Zor'], dtype='<U11')

In [36]:
np.random.choice(place, size=10, replace=False)

array(['Evicio', 'Im', 'Zelfher', 'Kesser', 'Birdwert', 'Swred', 'Tewral',
       'Tell', 'Yernall', 'Dine'], dtype='<U11')

In [37]:
np.random.choice(place, size=10, replace=False)

array(['Prige', 'Narg', 'Astral', 'Kenny', 'Undoros', 'Lights', 'Seller',
       'Wleur', 'Red', 'Geller'], dtype='<U11')

In [38]:
generate_names()

['Quan',
 'Wulfus',
 'Elphelt Valentine',
 'Rosfi Jease',
 'Toorike Adeboin',
 'Yorta Snekia',
 'Uleva',
 'Inaru',
 'Oreas Runsbol',
 'Priscile',
 'Ale',
 'Strait',
 'Damcie',
 'Fishtran',
 'Guberou',
 'Hrulysus',
 'Joacin',
 'Krace',
 'Leocaen',
 'Zunny',
 'Xashnighwall de Haillenarte',
 'Chars',
 'Valens',
 'Beldaix',
 'Noraa Misuri',
 'Motsobe']